# Collecting the required photos

In [74]:
import cv2
import matplotlib.pyplot as mpl
cap = cv2.VideoCapture(0)
count = 0
model = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
while True:
    ret, photo = cap.read()
    faces = model.detectMultiScale(photo)
    if len(faces) is None:
        print("Face not found")
        pass
    elif len(faces) is not None:
        count +=1
        x1 = faces[0][0]
        y1 = faces[0][1]
        x2 = x1 + faces[0][2]
        y2 = y1 + faces[0][3]
        
        #photor =  cv2.rectangle(photo, (x1,y1), (x2,y2), [0,255,0], 5)
        gray_photo = cv2.cvtColor(photo, cv2.COLOR_BGR2GRAY)
        cropped_photo = gray_photo[y1:y2, x1:x2]
        photor = cv2.resize(cropped_photo, (200,200))
        file_path = "./photos/"+ str(count)+"new.jpg"
        cv2.imwrite(file_path,photor)
        cv2.putText(photor, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow("photo", photor)
        if cv2.waitKey(1) ==13 or count==100:
            break
cv2.destroyAllWindows()
cap.release()
print("Collected required images")

Collected required images


# Training the model from above collected photos

In [75]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

# Get the training data we previously made
data_path = "./photos/"
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)

# Initialize facial recognizer
# model = cv2.face.createLBPHFaceRecognizer()
# NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()
# pip install opencv-contrib-python
# model = cv2.createLBPHFaceRecognizer()

Ranga_Mani_model = cv2.face_LBPHFaceRecognizer.create()
# Let's train our model 
Ranga_Mani_model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained sucessfully")
Ranga_Mani_model.save("rangamanimodel.h5")
print("Model saved successfully")


Model trained sucessfully
Model saved successfully


# Creating mail function for sending mail

In [76]:
def mail():
    import smtplib
    import os
    from email.message import EmailMessage
    msg = EmailMessage()
    user = %env USER
    password = %env PASSWD
    mailr = "ranga.mani54@gmail.com"
    msg.set_content("Hello Your face is recognised")
    msg["subject"] = "Face Recognition"
    msg["to"] = "ranga.mani54@gmail.com"
    msg["from"] = user
    server = smtplib.SMTP("smtp.gmail.com", 587)
    server.starttls()
    server.login(user, password)
    server.send_message(msg)
    print("Email sent successfully")

# Function for Whatsapp

In [77]:
def what():
    # importing the module
    import pywhatkit
    from datetime import datetime
    now = datetime.now()
    m = int(now.strftime("%M"))
    hr = int(now.strftime("%H"))

    # using Exception Handling to avoid
    # unprecedented errors
    try:

        # sending message to reciever
        # using pywhatkit
        number = %env NUMBER
        pywhatkit.sendwhatmsg(number,"Hello, face is recognised",hr, m+1, wait_time=30)
        print("Successfully Sent!")

    except:

        # handling exception
        # and printing error message
        print("An Unexpected Error!")


# Predicting the face and sending mail and whatsapp message

In [78]:
import cv2
import numpy as np
import os
import time
import threading


face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


# Open Webcam
cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Pass face to prediction model
        # "results" comprises of a tuple containing the label and the confidence value
        model = cv2.face_LBPHFaceRecognizer.create()
        model.read("rangamanimodel.h5")
        results = model.predict(face)
        # harry_model.predict(face)
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is User'
            
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        
        if confidence > 80:
            cv2.putText(image, "Hey Ranga Mani", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )
            t1 = threading.Thread(target=mail)
            t2 = threading.Thread(target = what)
            t1.start()
            t2.start()
            break
        else:
            cv2.putText(image, "I dont know, who r u", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )

    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
    
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      

<>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-78-c4ae867ff777>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


In 19 seconds web.whatsapp.com will open and after 30 seconds message will be delivered
Email sent successfully
Successfully Sent!


# Collecting friend's photos

In [ ]:
cap = cv2.VideoCapture(0)
count = 0
while True:
    ret, photo = cap.read()
    model = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    faces = model.detectMultiScale(photo)
    if len(faces) is None:
        print("Face not found")
        pass
    elif len(faces) is not None:
        count +=1
        x1 = faces[0][0]
        y1 = faces[0][1]
        x2 = x1 + faces[0][2]
        y2 = y1 + faces[0][3]
        
        #photor =  cv2.rectangle(photo, (x1,y1), (x2,y2), [0,255,0], 5)
        gray_photo = cv2.cvtColor(photo, cv2.COLOR_BGR2GRAY)
        cropped_photo = gray_photo[y1:y2, x1:x2]
        photor = cv2.resize(cropped_photo, (200,200))
        file_path = "./photos/friend/"+ str(count)+".jpg"
        cv2.imwrite(file_path,photor)
        cv2.putText(photor, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow("photo", photor)
        if cv2.waitKey(1) ==13 or count==100:
            break
cv2.destroyAllWindows()
cap.release()
print("Collected required images")

# Training the model for friend

In [86]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

# Get the training data we previously made
data_path = "./photos/friend/"
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)

# Initialize facial recognizer
# model = cv2.face.createLBPHFaceRecognizer()
# NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()
# pip install opencv-contrib-python
# model = cv2.createLBPHFaceRecognizer()

friend_model = cv2.face_LBPHFaceRecognizer.create()
# Let's train our model 
friend_model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained sucessfully")
friend_model.save("friend.h5")
print("Model saved successfully")


Model trained sucessfully
Model saved successfully


# Functions for Launching Instance, Volume and Attaching

In [87]:
def output(cmd):
    from subprocess import getstatusoutput
    global akey
    akey = getstatusoutput(cmd)
    if akey[0] == 0:
        print("Here is the output you are looking for")
        print(akey[1])
    else: 
        print("Error occured : {}".format(akey[1]))
    
            

# To launch Instance in AWS
def aws_linstances(instance_type,instance_number,sg_id,key_name,name):
    output("aws ec2 run-instances --image-id ami-0e306788ff2473ccb --instance-type {0} --count {1} --subnet-id subnet-c40108ac --security-group-ids {3} --key-name {2} --tag-specifications ResourceType=instance,Tags=[{{Key=Name,Value={4}}}]".format(
		instance_type,instance_number,key_name,sg_id,name))
    
            

# To create EBS Volume
def aws_cvol(size,name):
    output("aws ec2 create-volume --availability-zone ap-south-1a --volume-type gp2 --size {0} --tag-specifications ResourceType=volume,Tags=[{{Key=Name,Value={1}}}]".format(size,name))
    
            
# To Attach EBS Volume
def aws_avol(instance_id,vol_id):
    output("aws ec2 attach-volume --volume-id {0} --instance-id {1} --device /dev/sdf".format(vol_id,instance_id))
    

# Predicting the friend's face and doing the required action

In [88]:
import cv2
import numpy as np
import os
import time
import threading


face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


# Open Webcam
cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Pass face to prediction model
        # "results" comprises of a tuple containing the label and the confidence value
        model = cv2.face_LBPHFaceRecognizer.create()
        model.read("friend.h5")
        results = model.predict(face)
        # harry_model.predict(face)
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is User'
            
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        
        if confidence > 80:
            cv2.putText(image, "Hey friend", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )
            t1 = threading.Thread(target=aws_linstances, args=("t2.micro", 1,"sg-0f2a5a273a2a190ec", "hadoopkey", "testing"))
            t2 = threading.Thread(target = aws_cvol, args=(3,"testing"))
            t1.start()
            t2.start()
            inst_id = input("Enter Instance ID: ")
            vol_id = input("Enter Volume ID: ")
            aws_avol(inst_id, vol_id)
            break
        else:
            cv2.putText(image, "I dont know, who r u", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )

    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
    
    if cv2.waitKey(10) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      

<>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-88-b8c658816328>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


Here is the output you are looking for
{
    "AvailabilityZone": "ap-south-1a",
    "CreateTime": "2021-06-24T17:57:17+00:00",
    "Encrypted": false,
    "Size": 3,
    "SnapshotId": "",
    "State": "creating",
    "VolumeId": "vol-0bd15a1d6ace415a0",
    "Iops": 100,
    "Tags": [
        {
            "Key": "Name",
            "Value": "testing"
        }
    ],
    "VolumeType": "gp2",
    "MultiAttachEnabled": false
}
Here is the output you are looking for
{
    "Groups": [],
    "Instances": [
        {
            "AmiLaunchIndex": 0,
            "ImageId": "ami-0e306788ff2473ccb",
            "InstanceId": "i-0bb062a837ee09d06",
            "InstanceType": "t2.micro",
            "KeyName": "hadoopkey",
            "LaunchTime": "2021-06-24T17:57:17+00:00",
            "Monitoring": {
                "State": "disabled"
            },
            "Placement": {
                "AvailabilityZone": "ap-south-1a",
                "GroupName": "",
                "Tenancy": "defa